In [1]:
import pandas as pd
import numpy as np

import nltk
import os
import re

from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import NMF

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import pyLDAvis
import pyLDAvis.sklearn

from collections import Counter

import warnings
warnings.filterwarnings("ignore")

word_lm = WordNetLemmatizer()

#### Data: https://archive.ics.uci.edu/ml/datasets/Health+News+in+Twitter
#### using this file everydayhealth.txt

In [2]:
path = 'Health-Tweets'
col_names = ['tw_id', 'date', 'tweet']
df = pd.read_csv(os.path.join(path, 'everydayhealth.txt'), sep="|", names=col_names)
print len(df)
df.head()

3239


,tw_id,date,tweet
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a..."
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...


In [3]:
# some preprocessing

def pre_processing(data_frame, txt_clm):
    
    df_txt = data_frame.copy()
    
    # lower case 
    df_txt['lower_txt'] = df_txt[txt_clm].str.lower()
    
    #replace RTs with ''
    df_txt['links_rm'] = df_txt['lower_txt'].apply(lambda v: v.replace('rt', ''))
    
    # remove punctuation
    df_txt['punc_rm'] = df_txt['lower_txt'].str.replace('[^\w\s]','')
    
    # replace hyperlinks with ''
    df_txt['links_rm'] = df_txt['punc_rm'].apply(lambda v: re.sub(r'http\S+', '', v))
    
    # remove stopwords
    df_txt['stopwords_rm'] = df_txt['links_rm'].apply(lambda x: " ".join(x for x in x.split() if x not in ENGLISH_STOP_WORDS))
    
     # lemmatize all the words
    df_txt['lemmatize_text'] = df_txt['stopwords_rm'].apply(lambda v: " ".join([word_lm.lemmatize(i) for i in v.split()]))
    
    df_txt['processed_text'] = df_txt['lemmatize_text'].apply(lambda x: word_tokenize(x))
    
    df_txt['raw_text'] = df_txt['processed_text'].apply(lambda x: " ".join(w for w in x if len(w)> 3))
    
    return df_txt[['tweet', 'raw_text']]

tweets_data = df.copy()
processed_df = pre_processing(tweets_data[['tweet']], 'tweet')
processed_df['raw_text'].tolist()[:10]

[u'fastfood make percent calorie diet',
 u'snack help lose weight burn build muscle',
 u'food boost skin slim waistline',
 u'heart attack feel like woman different gored',
 u'mcdonalds oatmeal teaspoon sugar healthy fast food isnt',
 u'food boost skin slim waistline',
 u'skipping kegels using talcum powder vaginal health mistake youre probably making',
 u'food boost skin slim waistline',
 u'today happier perform random kindness itll make feel better',
 u'depression isnt everybody different face disorder']

In [4]:
# try a groupby of raw text, since preprocessing now we are able to group them
group_text = processed_df.groupby(['raw_text'], as_index=False).count()
group_text = group_text.rename(columns={'tweet': 'count'})
group_text.sort_values('count', ascending=False).head(10)

,raw_text,count
725,everyday health daily digest,42
817,food boost skin slim waistline,24
824,food longer life,22
542,eating habit pack pound,14
1853,soda drinker listen reason kick soda habit,12
1353,meat surprising food thatll spike blood sugar,11
785,fiberrich food diet asap,11
247,calorie today,10
14,adhd characteristic actually huge plus choosin...,10
968,health symptom shouldnt ignore menshealth,9


In [5]:
cnt_vectorizer = CountVectorizer(stop_words='english')
data_vectorized = cnt_vectorizer.fit_transform(processed_df['raw_text'])
cnt_ftrs_names = cnt_vectorizer.get_feature_names()
cnt_ftrs_df = pd.DataFrame(data_vectorized.toarray(), columns=cnt_ftrs_names)
cnt_ftrs_df.head(3)

,1000,1089,11am,11ampst,1200,12noon,13for13,13in2013,1520lbs,1pmet,...,youself,youth,youtube,youve,yummy,zero,zerocalorie,zocor,zucchini,zzzs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(processed_df['raw_text'])
feature_names = vectorizer.get_feature_names() # num phrases 
feature_df = pd.DataFrame(X.toarray(), columns=feature_names)
feature_df.head(3)

,1000,1089,11am,11ampst,1200,12noon,13for13,13in2013,1520lbs,1pmet,...,youself,youth,youtube,youve,yummy,zero,zerocalorie,zocor,zucchini,zzzs
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# get top n features
feature_array = np.array(feature_names)
tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]

n = 30
top_n = feature_array[tfidf_sorting][:n]
print top_n

[u'worry' u'sooner' u'highincome' u'sicker' u'country' u'american' u'zzzs'
 u'fightdepression' u'finger' u'finding' u'financial' u'finally'
 u'filtered' u'filling' u'figure' u'fighting' u'fiery' u'fight'
 u'fitandfabliving' u'field' u'fibromyalgia' u'fibrillation' u'fiberrich'
 u'fiber' u'fewer' u'fever' u'feng' u'fend' u'fence' u'fish']


In [8]:
# apply k means cluster, choosing k as 7
true_k = 7
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)
clusters = model.labels_.tolist()
cluster_items = Counter(clusters)
for k in cluster_items:
    print "Cluster {}: {}".format(k, cluster_items[k])

Cluster 0: 1861
Cluster 1: 175
Cluster 2: 670
Cluster 3: 102
Cluster 4: 42
Cluster 5: 241
Cluster 6: 148


In [9]:
#sort cluster centers by proximity to centroid
cl_cols = ['Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4', 'Cluster 5', 'Cluster 6']
cluster_df = pd.DataFrame(columns = cl_cols)

order_centroids = model.cluster_centers_.argsort()[:, ::-1]

for i in range(true_k):
    join_words = []
    for ind in order_centroids[i, :20]:
        join_words.append(feature_names[ind])
    cluster_df["Cluster {}".format(i)] = join_words
    
cluster_df.head(10)

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6
0,calorie,eatsmartbd,healthtalk,reason,digest,food,weight
1,health,healthtalk,everydayhealth,soda,everyday,life,lose
2,exercise,time,join,fish,daily,longer,loss
3,better,everydayhealth,psoriasis,lifeenhancing,health,skin,gain
4,really,love,today,losing,fightdepression,boost,whats
5,today,follow,heart,drinker,fitbit,slim,pound
6,jillianmichaels,chat,vegan,kick,fitandfabliving,waistline,dieting
7,diet,menshealth,question,listen,fish,sugar,trying
8,help,food,welcome,chocolate,finger,spike,healthy
9,eating,like,rmichlermd,youre,finding,blood,help


In [10]:
topic_cols = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6']
topics_df = pd.DataFrame(columns=topic_cols)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        
        top_words = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            top_words.append(feature_names[i])
                             
        topics_df["Topic {}".format(topic_idx)] = top_words
    return topics_df
        
    
        
# Run NMF
no_topics = 7
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(X)

no_top_words = 20
display_topics(nmf, feature_names, no_top_words).head(10)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,digest,food,healthtalk,weight,calorie,boost,eating
1,everyday,life,join,lose,today,waistline,habit
2,daily,longer,everydayhealth,loss,burn,slim,pound
3,health,change,follow,help,chocolate,skin,pack
4,worst,worst,eatsmartbd,whats,easy,food,healthy
5,snack,sugar,heart,healthy,sneaky,best,soda
6,ignore,spike,chat,trying,just,blood,reason
7,shouldnt,blood,vegan,reason,reason,sugar,complexion
8,2013,asap,question,dieting,skip,plus,drinker
9,symptom,fiberrich,today,losing,tomorrow,metabolism,kick


In [11]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=7, learning_method='online')
lda_model

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=7, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [12]:
lda_output = lda_model.fit_transform(data_vectorized)
display_topics(lda_model, cnt_ftrs_names, no_top_words).head(10)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6
0,calorie,reason,food,healthtalk,weight,help,healthtalk
1,make,eating,life,people,health,cancer,everydayhealth
2,study,habit,boost,need,jillianmichaels,share,healthy
3,burn,pound,diet,best,lose,great,today
4,chocolate,secret,worst,really,exercise,read,eatsmartbd
5,easy,pack,surprising,risk,daily,whats,join
6,workout,test,sugar,diabetes,want,fight,good
7,thanks,fish,soda,answer,everyday,stop,time
8,making,biggest,blood,question,youre,love,chat
9,drug,real,skin,know,loss,exercise,love


In [13]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, cnt_vectorizer, mds='tsne')
panel

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
6      21.273944        1       1 -337.663269  214.850662
3      17.978852        1       2   62.527966  408.452667
4      15.116467        1       3  397.847656 -284.704498
2      13.844595        1       4   -2.353624 -478.320221
0      11.305713        1       5  430.290894  158.681808
5      10.851559        1       6 -370.111206 -228.526093
1       9.628871        1       7   30.093769  -34.935024, topic_info=     Category        Freq             Term       Total  loglift  logprob
term                                                                    
1429  Default  742.000000       healthtalk  742.000000  30.0000  30.0000
1221  Default  279.000000             food  279.000000  29.0000  29.0000
3424  Default  137.000000           weight  137.000000  28.0000  28.0000
441   Default  101.000000          calorie  101.000000  27.0000  27.0000
1432  Default  143.000000          healthy  143.000000  26.0000  26.0000
1419  Default  176.000000           health  176.000000  25.0000  25.0000
2526  Default   87.000000           reason   87.000000  24.0000  24.0000
969   Default   86.000000           eating   86.000000  23.0000  23.0000
1065  Default  175.000000   everydayhealth  175.000000  22.0000  22.0000
3193  Default  121.000000            today  121.000000  21.0000  21.0000
1788  Default   84.000000             life   84.000000  20.0000  20.0000
1460  Default  110.000000             help  110.000000  19.0000  19.0000
448   Default   63.000000           cancer   63.000000  18.0000  18.0000
1669  Default   68.000000  jillianmichaels   68.000000  17.0000  17.0000
356   Default   64.000000            boost   64.000000  16.0000  16.0000
1378  Default   45.000000            habit   45.000000  15.0000  15.0000
975   Default  110.000000       eatsmartbd  110.000000  14.0000  14.0000
1846  Default   54.000000             lose   54.000000  13.0000  13.0000
2273  Default   57.000000           people   57.000000  12.0000  12.0000
1325  Default   63.000000             good   63.000000  11.0000  11.0000
3185  Default   61.000000             time   61.000000  10.0000  10.0000
3497  Default   46.000000            worst   46.000000   9.0000   9.0000
3061  Default   46.000000       surprising   46.000000   8.0000   8.0000
1856  Default   76.000000             love   76.000000   7.0000   7.0000
764   Default   47.000000            daily   47.000000   6.0000   6.0000
2769  Default   39.000000            share   39.000000   5.0000   5.0000
1080  Default   90.000000         exercise   90.000000   4.0000   4.0000
3392  Default   45.000000             want   45.000000   3.0000   3.0000
1064  Default   45.000000         everyday   45.000000   2.0000   2.0000
515   Default   56.000000             chat   56.000000   1.0000   1.0000
...       ...         ...              ...         ...      ...      ...
298    Topic7   18.412524          biggest   19.144272   2.3014  -4.6409
2519   Topic7   15.600876             real   16.333261   2.2945  -4.8066
723    Topic7   15.439894          craving   16.171759   2.2941  -4.8170
2068   Topic7   13.920891             myth   14.653016   2.2891  -4.9206
1815   Topic7   13.544761           listen   14.277319   2.2877  -4.9479
2218   Topic7   12.631377             p90x   13.362256   2.2842  -5.0178
751    Topic7   11.862966             cure   12.593956   2.2806  -5.0805
3264   Topic7   11.754901            truth   12.486337   2.2800  -5.0897
3536   Topic7   11.285452            youve   12.016634   2.2776  -5.1304
3370   Topic7   10.696474          vitamin   11.428266   2.2742  -5.1840
1707   Topic7   10.366794             kick   11.098351   2.2722  -5.2153
1552   Topic7   10.257310           impact   10.989803   2.2714  -5.2260
420    Topic7    9.832180           butter   10.564482   2.2686  -5.2683
2288   Topic7    9.811299      personality   10.542941   2.2685  -5.2704
933 